In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.__version__

In [ ]:
df = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')
df.head()

# Part1: Feature Eengineering

### 1-Checking for missing data and recognize the features

In [ ]:
df.info()

### 2-Exclude the non-relational features: 'RowNumber'  'CustomerId'  'Surname'

In [ ]:
X = df.iloc[:, 3: -1].values
y = df.iloc[:, -1].values
print(X.shape)
y

### 3-Apply label encoding to Gender column (nominal encoding)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:,2])
X

### 4-Apply OneHotEncoder to the 'Geography' column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

### 5-Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
X_test.shape

### 6-Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

# Part2: Building ANN

In [ ]:
ann = tf.keras.models.Sequential()
type(ann)

### Adding the input layer, the scond hidden Layer and the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compile the ann by specify loss and optimizer

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs= 100)

In [ ]:
ann.summary()

In [ ]:
X_test

In [ ]:
y_pred = ann.predict(X_test)
y_pred.shape

In [ ]:
y_pred = y_pred>0.5
y_pred[:15]

In [ ]:
# concatenate predictions with test
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
accuracy_score(y_test, y_pred)

### THis is a very goog accuracy cause its almost the same with the training set!

### Use our ANN model to predict if the customer with the following informations will leave the bank:

### Geography: France

### Credit Score: 600

### Gender: Male

### Age: 40 years old

### Tenure: 3 years

### Balance: $ 60000

### Number of Products: 2

### Does this customer have a credit card? Yes

### Is this customer an Active Member: Yes

### Estimated Salary: $ 50000

### So, should we say goodbye to that customer?

In [ ]:
X[0]

In [ ]:
[1,0,0,600,1,40,3,60000,2,1,1,50000]

In [ ]:
sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])

In [ ]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))

### The probability of the customer stays in the bank is 0,02 so he will leave the bank probably

In [ ]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5